In [2]:
import whisper
import datasets
import pydub
import soundfile
import pandas as pd
import numpy as np
import librosa
import os
from pydub import AudioSegment

In [3]:
dataset = datasets.load_dataset("CAiRE/ASCEND")

In [4]:
# Function to separate mp3 file by timestamp
set = pd.DataFrame(dataset['train'])
display(set)

,id,path,audio,transcription,duration,language,original_speaker_id,session_id,topic
0,00000,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,我刚刚开始record,1.560,mixed,1,1,persona
1,00001,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,嗯hello我的名字叫徐妍,4.160,mixed,1,1,persona
2,00002,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,嗯初次见面nice to meet you嗯,3.320,mixed,1,1,persona
3,00003,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,今天呢我非常希望能够通过这个机会去跟你make friends,5.700,mixed,1,1,persona
4,00004,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,嗯你知道就是,2.020,zh,1,1,persona
...,...,...,...,...,...,...,...,...,...
9864,09864,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,嗯,0.550,zh,8,4,technology
9865,09865,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,her intellect intelli,1.485,en,8,3,education
9866,09866,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,学术方面的话 其实国内也有很多 学术很厉害 的 嗯 professor之类的 但是可能整体的...,14.506,mixed,8,3,education
9867,09867,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,home school education can provide an environme...,8.611,en,8,3,education


In [5]:
path = set.at[4701, 'path']
print(path)

C:\Users\saleh\.cache\huggingface\datasets\downloads\extracted\f7ac516b8b52e26639eabec77d98278b293cb5a5579773126b8901eb92fb8307\waves/ses2_spk24_L12230_727.840_0.680.wav


In [60]:
total_wav = AudioSegment.from_wav(path)
total_wav_file = total_wav.export("total_wav.wav", format="wav")
duration = len(total_wav)
model = whisper.load_model("base")

def split_audio(file, start, end):
    wav = AudioSegment.from_wav(file)
    first_half = wav[start:end]
    second_half = wav[end:]
    first_half.export("first_half.wav", format="wav")
    second_half.export("second_half.wav", format="wav")

def get_highest_whisper_entry(file):
    audio = whisper.load_audio(file)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
    sorted_probs = [{'key': key, 'value': value} for key, value in sorted_probs]
    return sorted_probs[0]

my_dict = {}
def binary_search(file, start, end, new_file):
    if get_highest_whisper_entry(new_file)['value'] > 0.9:
        my_dict[start] = get_highest_whisper_entry(new_file)['key']
        print('high prob of', get_highest_whisper_entry(new_file)['value'] ,'that its 1 language:', my_dict)
        
        return
    
    if end - start <= 1000:
        my_dict[start] = get_highest_whisper_entry(new_file)['key']
        print('clip too short, split:', my_dict)
        return
    
    split_position = start + np.ceil((end - start) / 2)
    split_position = int(split_position)
    split_audio(file, start, split_position)
    binary_search(file, start, split_position, "first_half.wav")
    
    # Update split position for the second half
    split_position = int(split_position)
    split_audio(file, split_position, end)
    binary_search(file, split_position, end, "second_half.wav")

binary_search("total_wav.wav", 0, len(AudioSegment.from_wav("total_wav.wav")), "total_wav.wav")
print(my_dict)


high prob of 0.9799582362174988 that its 1 language: {0: 'zh'}
{0: 'zh'}


In [ ]:
def translate_audio(audio_path, segments_dict, model_name="base"):
    model = whisper.load_model(model_name)
    full_audio = AudioSegment.from_file(audio_path, format="wav")

    translated_texts = []
    start_time = 0  # Start of the first segment

    for end_time, language in segments_dict.items():
        start_ms = start_time * 1000
        end_ms = end_time * 1000

        audio_segment = full_audio[start_ms:end_ms]
        temp_file = "temp_segment.wav"
        audio_segment.export(temp_file, format="wav")

        result = model.transcribe(temp_file, task="translate", language=language)
        translated_texts.append(result['text'])

        os.remove(temp_file)
        start_time = end_time

    full_translation = ' '.join(translated_texts)
    return full_translation

In [58]:
# Example of how to use Whisper to transcribe/translate an audio file
# Split dictionary output above into separate audio files and transcribe separately


model = whisper.load_model("base")
audio = whisper.load_audio("total_wav.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)

_, probs = model.detect_language(mel)

# Sorting the dictionary items by values in descending order
sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)

# Creating a new list with the sorted key-value pairs
sorted_probs = [{'key': key, 'value': value} for key, value in sorted_probs]

print(sorted_probs)

options = whisper.DecodingOptions(task='transcribe', fp16=False)
result = whisper.decode(model, mel, options)
print(result.text)

options = whisper.DecodingOptions(task='translate', fp16=False)
result = whisper.decode(model, mel, options)
print(result.text)

[{'key': 'zh', 'value': 0.8531141877174377}, {'key': 'en', 'value': 0.12959668040275574}, {'key': 'ja', 'value': 0.004206022247672081}, {'key': 'ko', 'value': 0.00377671024762094}, {'key': 'th', 'value': 0.002794167259708047}, {'key': 'vi', 'value': 0.0013848075177520514}, {'key': 'ms', 'value': 0.0005554488743655384}, {'key': 'my', 'value': 0.0005255815922282636}, {'key': 'nn', 'value': 0.0003939943271689117}, {'key': 'id', 'value': 0.000368792621884495}, {'key': 'ru', 'value': 0.00035734925768338144}, {'key': 'es', 'value': 0.00035443922388367355}, {'key': 'la', 'value': 0.00032263994216918945}, {'key': 'km', 'value': 0.00026765474467538297}, {'key': 'jw', 'value': 0.00023231747036334127}, {'key': 'de', 'value': 0.00020260077144484967}, {'key': 'cy', 'value': 0.00018425822781864554}, {'key': 'tl', 'value': 0.0001657976972637698}, {'key': 'haw', 'value': 0.00015468480705749243}, {'key': 'pl', 'value': 0.00013227536692284048}, {'key': 'pt', 'value': 0.0001043591764755547}, {'key': 'fr'